In [9]:
import matplotlib.pyplot as plt
import pandas as pd
import requests as req
from statsmodels.graphics.tsaplots import quarter_plot
import numpy as np
from matplotlib.backends.backend_pdf import PdfPages



AttributeError: module 'numpy' has no attribute 'MachAr'

1.24.3


In [ ]:
url = 'https://apisidra.ibge.gov.br/values/t/5932/n1/all/v/6561/p/all/c11255/90687,90691,90696,90707,93404,93405,93406,93407,93408/d/v6561%201'



r = req.get(url)

raw = r.json()
df = pd.DataFrame(raw)
df.columns = df.iloc[0]
df.drop(0, inplace=True)
df.drop(df.columns[[0,1,2,3,5,6,7,8,10,11]], axis=1, inplace=True)
df['Trimestre (Código)'] = df['Trimestre (Código)'].map(str).str[:4] + 'Q' + df['Trimestre (Código)'].map(str).str[-1]
df = df.apply(pd.to_numeric, errors='ignore')
df['Trimestre'] = pd.to_datetime(df['Trimestre (Código)']).dt.to_period('Q')
df['Trimestre (string)'] = df['Trimestre'].astype('str')

set_list = list(df['Setores e subsetores'].unique())

pp = PdfPages(r"C:\Users\mkenj\Desktop\Github repositories\IPCA\images\teste_pib_var_QoQ.pdf")

for setor in set_list:
    plt.figure(figsize=(11.69,8.27))
    data = df[['Valor', 'Setores e subsetores', 'Trimestre', 'Trimestre (string)']]
    line = data[data['Setores e subsetores']==setor]
    line.reset_index(inplace=True, drop=True)
    plt.plot(line['Trimestre (string)'], line['Valor'])
    locs, labels = plt.xticks()
    ticks_positions = list(np.arange(locs[0], locs[-1],20))
    plt.xticks(ticks_positions, line['Trimestre (string)'][ticks_positions])
    plt.grid(axis='y')
    plt.annotate('fonte: IBGE, elaboração: Safra Asset', (0, 0), (0, -30),
    fontsize=10, xycoords='axes fraction', textcoords='offset points', va='top')
    plt.title(setor, fontsize=14)
    pp.savefig(orientation = 'landscape')
    plt.close()
    quarter_line = line.set_index('Trimestre')
    quarter_plot(quarter_line.Valor)
    plt.gcf().set_size_inches(11.69,8.27)
    trimestres = ['1ºTri', '2ºTri', '3ºTri', '4ºTri']
    plt.xticks(plt.xticks()[0], trimestres)
    plt.xlim((-5, len(line) + 5))
    bottom, top = plt.ylim()
    titulo = setor + ' (Gráfico de sazonalidade)'
    plt.title(titulo, fontsize=14)
    plt.grid(axis='y')
    plt.annotate('fonte: IBGE, elaboração: Safra Asset', (0,0), (0,-30),
    fontsize = 10, xycoords = 'axes fraction', textcoords = 'offset points', va = 'top')